## Test onnx-tf convertor
This notebook serves to test if the onnx model contains the architecture of keras/tf model <br>
and whether the saved onnx model can be used to reconstruct the keras/tf model.

### Build keras/tf model from scratch
loading dependency and prepare data

In [1]:
# this is the code from https://github.com/keras-team/keras/blob/master/examples/mnist_cnn.py
from __future__ import print_function
from tensorflow import keras
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras import backend as K

batch_size = 128
num_classes = 10
epochs = 2

# input image dimensions
img_rows, img_cols = 28, 28

# the data, split between train and test sets
(x_train, y_train), (x_test, y_test) = mnist.load_data()

if K.image_data_format() == 'channels_first':
    x_train = x_train.reshape(x_train.shape[0], 1, img_rows, img_cols)
    x_test = x_test.reshape(x_test.shape[0], 1, img_rows, img_cols)
    input_shape = (1, img_rows, img_cols)
else:
    x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
    x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
    input_shape = (img_rows, img_cols, 1)

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

x_train shape: (60000, 28, 28, 1)
60000 train samples
10000 test samples


### Create network

In [2]:
model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3),
                 activation='relu',
                 input_shape=input_shape))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))

model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])

2021-11-18 10:18:01.139684: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2021-11-18 10:18:01.140175: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2021-11-18 10:18:01.144851: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.


### Start training and evaluate model afterwards

In [3]:
model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(x_test, y_test))
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

2021-11-18 10:18:03.469320: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:116] None of the MLIR optimization passes are enabled (registered 2)
2021-11-18 10:18:03.486253: I tensorflow/core/platform/profile_utils/cpu_utils.cc:112] CPU Frequency: 2304005000 Hz


Epoch 1/2
469/469 [==============================] - 280s 591ms/step - loss: 2.2937 - accuracy: 0.1228 - val_loss: 2.2412 - val_accuracy: 0.3340
Epoch 2/2
469/469 [==============================] - 159s 339ms/step - loss: 2.2340 - accuracy: 0.2535 - val_loss: 2.1654 - val_accuracy: 0.5650
Test loss: 2.1653730869293213
Test accuracy: 0.5649999976158142


### Save keras/tf model

In [4]:
import tensorflow as tf

savedmodel_dir = 'mnist_savedmodel'
tf.saved_model.save(model, savedmodel_dir)

2021-11-18 10:29:19.880077: W tensorflow/python/util/util.cc:348] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.


INFO:tensorflow:Assets written to: mnist_savedmodel/assets


### Convert keras/tf model to onnx format

In [7]:
import onnx
import onnxruntime as ort
from onnx_tf.backend import prepare
# SavedModel to onnx
onnx_savedmodel = 'mnist_savedmodel.onnx'
!python -m tf2onnx.convert --saved-model {savedmodel_dir} --output {onnx_savedmodel} --signature_def serving_default --tag serve

/home/yangliu/miniconda3/lib/python3.8/runpy.py:127: RuntimeWarning: 'tf2onnx.convert' found in sys.modules after import of package 'tf2onnx', but prior to execution of 'tf2onnx.convert'; this may result in unpredictable behaviour
  warn(RuntimeWarning(msg))
2021-11-18 10:34:42,434 - INFO - Signatures found in model: [serving_default].
2021-11-18 10:34:42,434 - INFO - Output names: ['dense_1']
Instructions for updating:
Use `tf.compat.v1.graph_util.extract_sub_graph`
2021-11-18 10:34:42,600 - WARNING - From /home/yangliu/miniconda3/lib/python3.8/site-packages/tf2onnx/tf_loader.py:706: extract_sub_graph (from tensorflow.python.framework.graph_util_impl) is deprecated and will be removed in a future version.
Instructions for updating:
Use `tf.compat.v1.graph_util.extract_sub_graph`
2021-11-18 10:34:42,814 - INFO - Using tensorflow=2.4.1, onnx=1.9.0, tf2onnx=1.9.3/1190aa
2021-11-18 10:34:42,815 - INFO - Using opset <onnx, 9>
2021-11-18 10:34:43,125 - INFO - Computed 0 values for constant 

### Load saved onnx model and convert it back to keras/tf model

In [8]:
onnx_model = onnx.load("mnist_savedmodel.onnx")  # load onnx model
tf_model_rep = prepare(onnx_model)
tf_model_rep.export_graph("mnist_model_graph")
tf_model = tf.saved_model.load("mnist_model_graph")

/home/yangliu/miniconda3/lib/python3.8/site-packages/tensorflow_addons/utils/ensure_tf_install.py:53: UserWarning: Tensorflow Addons supports using Python ops for all Tensorflow versions above or equal to 2.5.0 and strictly below 2.8.0 (nightly versions are not supported). 
 The versions of TensorFlow you are currently using is 2.4.1 and is not supported. 
Some things might work, some things might not.
If you were to encounter a bug, do not file an issue.
If you want to make sure you're using a tested and supported configuration, either change the TensorFlow version or the TensorFlow Addons's version. 
You can find the compatibility matrix in TensorFlow Addon's readme:
https://github.com/tensorflow/addons
  warnings.warn(


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Index'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Index'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


INFO:tensorflow:Assets written to: mnist_model_graph/assets


INFO:tensorflow:Assets written to: mnist_model_graph/assets


### Check if the architecture of keras/tf model can be printed out

In [9]:
tf_model.summary()

AttributeError: '_UserObject' object has no attribute 'summary'

### Check if shap is able to work with the loaded keras/tf model

In [10]:
import shap
import numpy as np
# shap using tf model
e = shap.DeepExplainer(tf_model, 0)

Using TensorFlow backend.
keras is no longer supported, please use tf.keras instead.
Your TensorFlow version is newer than 2.4.0 and so graph support has been removed in eager mode. See PR #1483 for discussion.


AssertionError: <class 'tensorflow.python.saved_model.load.Loader._recreate_base_user_object.<locals>._UserObject'> is not currently a supported model type!